---


#  🟢 **Minicurso** - Processamento e Visualização de Imagens de Satélite, Dados de Precipitação e Índices de Vegetação com Python (PyVisSat)

---

> ## **Aula Bonus:** $\underline{Trabalhando\ com\ Dados\ do\ ERA5\ na\ Nuvem}$

---
**OBJETIVO:**

- Nesta aula prática aprenderemos como carregar e processar os dados do [ERA5](https://github.com/google-research/arco-era5?tab=readme-ov-file) que estão disponíveis na nuvem.

---

**DADOS DE ENTRADA**:
- São dados globais de 273 variáveis meteorológicas em níveis de pressão (37 níveis) e superfície com resolução espacial horizontal de 25 km.  


---

**DADOS DE SAÍDA:**
- Serão produzidas figuras no formato PNG.

---

**PROCEDIMENTO REALIZADO:**
- Os seguintes procedimentos são realizados nessa aula:

    1.   **1° Passo:** Instalação e Importação das Bibliotecas
    3.   **PARTE 1)**: Carregando os Dados
    4.   **PARTE 2)**: Mapas Simples
    5.   **PARTE 3)**: Série Temporal
    6.   **PARTE 4)**: Mapa de Temperatura Mínima e Máxima


---
**OBSERVAÇÕES IMPORTANTES**:
1. Salvar o código no seu drive. Para isto clicar em `Arquivo` e depois em `Salvar uma cópia no drive` e fazer o login numa conta google.

---

**REALIZADO E MINISTRADO POR:**
- **Realizado por:** Prof. Dr. Enrique V. Mattos - 24/11/2025
- **Atualizado por:** Prof. Dr. Enrique V. Mattos - 24/11/2025
---

# **1° Passo:** Instalação e Importação de Bibliotecas

In [ ]:
# instala bibliotecas
!pip install -q zarr gcsfs salem rasterio ultraplot cartopy

# importa bibliotecas
import salem
import xarray as xr
import ultraplot as uplt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# limites do Brasil
lonmin_BR, lonmax_BR, latmin_BR, latmax_BR = -75.0, -33.0, -35.0, 7.0

# leitura do shapefile do contorno do Brasil
shapefile_brasil = salem.read_shapefile('https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/BRAZIL.shp')

# leitura do shapefile dos estados
shapefile_estados = salem.read_shapefile('https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/BR_UF_2019.shp')

#**PARTE 1)**: Carregando os Dados

Os dados do ERA5 possui uma versão alocada na nuvem em formato Zarr, chamada de [arco-era5](https://github.com/google-research/arco-era5?tab=readme-ov-file) que facilita o carregamento e processamento de diversas variáveis meteorológicas. Com ele podemos acessar diversas variáveis do ERA5 em segundos, sem ter que baixar em nosso computador nenhum dado (*que via de regra levaria horas ou dias!*). Isto otimiza o processamento dos dados do ERA5, sobrando tempo para focarmos nas análises dos dados.

São dados globais horários de `273 variáveis` com resolução espacial horizontal de 25 km desde 1940. Algumas variáveis estão em níveis de pressão (37 níveis de pressão, 1 a 1000 hPa) ou superfície.

In [ ]:
# carrega os dados
era5_3d_1h = xr.open_zarr('gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
                          chunks=None,
                          storage_options=dict(token='anon'))

# transforma longitude de 0 e -360 graus para -180 e +180 graus
era5_3d_1h.coords['longitude'] = ((era5_3d_1h.coords['longitude'] + 180) % 360) - 180
era5_3d_1h = era5_3d_1h.sortby(era5_3d_1h.longitude)

Mostra os dados

In [ ]:
era5_3d_1h

Variáveis  

In [ ]:
era5_3d_1h.data_vars

Quantidade de variáveis disponíveis

In [ ]:
len(era5_3d_1h.data_vars)

Período dos dados que temos disponível

In [ ]:
era5_3d_1h.attrs

Níveis de pressão

In [ ]:
era5_3d_1h.level

Latitudes

In [ ]:
era5_3d_1h.latitude

Longitudes

In [ ]:
era5_3d_1h.longitude

#**PARTE 2)**: Mapas Simples

Acessando a temperatura a 2 m

In [ ]:
era5_3d_1h['2m_temperature']

Produz o mapa global de temperatura para uma data específica:
- 15 de novembro de 2025 às 15 UTC

In [ ]:
era5_3d_1h['2m_temperature'].sel(time='2025-11-15T15').plot(cmap='jet')

Produz o mapa de temperatura máxima de janeiro de 2025 para o Brasil

In [ ]:
# faz o recorte nos dados
era5_3d_1h['2m_temperature'].sel(time=slice('2025-01-01','2025-01-31'),
                                 longitude=slice(lonmin_BR, lonmax_BR),
                                 latitude=slice(latmax_BR, latmin_BR)).max('time').salem.roi(shape=shapefile_brasil).plot(cmap='jet')

# salva a figura
plt.savefig('Parte_2_mapa_simples_temperatura_max_jan_2025_ERA5.png', dpi=300, bbox_inches='tight')

#**PARTE 3)**: Série Temporal

Recorta temporalmente os dados para janeiro de 2025

In [ ]:
t2m_jan_2025 = era5_3d_1h['2m_temperature'].sel(time=slice('2025-01-01','2025-01-31'))

Extrai a série temporal de temperatura diária para uma localidade e plota a figura

In [ ]:
# extrai a série temporal
t2m_jan_2025.sel(longitude = -40,
                 latitude = -20.0,
                 method = 'nearest').plot()

# salva a figura
plt.savefig('Parte_3_serie_temporal_temperatura_diaria_jan2025_ERA5.png', dpi=300, bbox_inches='tight')

#**PARTE 4)**: Mapa de Temperatura Mínima e Máxima

Calculando a temperatura **mínima** e **máxima** para o dia `15 de novembro de 2025`

In [ ]:
# escolha a data
data = '2025-11-15'

# calculando a temperatura MÍNIMA
t2m_min = era5_3d_1h['2m_temperature'].sel(time=slice(data, data),
                                           longitude=slice(lonmin_BR, lonmax_BR),
                                           latitude=slice(latmax_BR, latmin_BR)).min('time').salem.roi(shape=shapefile_brasil)
# calculando a temperatura MÁXIMA
t2m_max = era5_3d_1h['2m_temperature'].sel(time=slice(data, data),
                                           longitude=slice(lonmin_BR, lonmax_BR),
                                           latitude=slice(latmax_BR, latmin_BR)).max('time').salem.roi(shape=shapefile_brasil)

Plotando o mapa

In [ ]:
# cria a moldura da figura
fig, ax = uplt.subplots(axwidth=6, ncols=2, nrows=1, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=False, latlines=5, lonlines=10,
          latlim=(latmin_BR, latmax_BR), lonlim=(lonmin_BR, lonmax_BR),
          small='20px', large='25px',
          linewidth=0, grid=False,
          suptitle=f'Temperatura Diária ($\degree$C)\n({data})')

#--------------------------------------------------------------#
#             FIGURA 1: TEMPERATURA MÍNIMA
#--------------------------------------------------------------#
# plota mapa
map1 = ax[0].contourf(t2m_min['longitude'],
                      t2m_min['latitude'],
                      t2m_min - 273.15,
                      cmap='coolwarm',
                      vmin=20, vmax=40,
                      levels=uplt.arange(20, 40, 2),
                      extend='both',
                      colorbar='best',
                      colorbar_kw={'label': 'Fonte: ERA5/Pixel: 25km',
                                   'length': 18,
                                   'frameon': False,
                                   'ticklabelsize': 10,
                                   'labelsize': 7,
                                   'width': 2,
                                   'ticks': 2,
                                   'minorticks': False})

# plota contorno dos Estados
shapefile_estados.plot(edgecolor='gray', facecolor='none', linewidth=0.5, alpha=1, ax=ax[0])

# plota contorno do Brasil
shapefile_brasil.plot(edgecolor='black', facecolor='none', linewidth=1.0, alpha=1, ax=ax[0])

# título da figura
ax[0].format(title='Mínima',
             titleloc='c',
             titleweight='bold',
             titlecolor='gray')

#--------------------------------------------------------------#
#              FIGURA 2: TEMPERATURA MÁXIMA
#--------------------------------------------------------------#
# plota mapa
map2 = ax[1].contourf(t2m_max['longitude'],
                      t2m_max['latitude'],
                      t2m_max - 273.15,
                      cmap='coolwarm',
                      vmin=20, vmax=40,
                      levels=uplt.arange(20, 40, 2),
                      extend='both',
                      colorbar='best',
                      colorbar_kw={'label': 'Fonte: ERA5/Pixel: 25km',
                                   'length': 18,
                                   'frameon': False,
                                   'ticklabelsize': 10,
                                   'labelsize': 7,
                                   'width': 2,
                                   'ticks': 2,
                                   'minorticks': False})

# plota contorno dos Estados
shapefile_estados.plot(edgecolor='gray', facecolor='none', linewidth=0.5, alpha=1, ax=ax[1])

# plota contorno do Brasil
shapefile_brasil.plot(edgecolor='black', facecolor='none', linewidth=1.0, alpha=1, ax=ax[1])

# título da figura
ax[1].format(title='Máxima',
             titleloc='c',
             titleweight='bold',
             titlecolor='gray')

# salva figura
fig.save('Parte_4_temperatura_diaria_ERA5.png', dpi=300)